<a href="https://colab.research.google.com/github/Birdnation/Rubik-2x2-SI/blob/main/TallerRubik2x2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 !cp "/content/drive/MyDrive/SistemaInteligente/lab2/utils.py" .

In [ ]:
import sys
from collections import deque
import time
from utils import *
import numpy as np


In [ ]:
class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [ ]:
class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state. Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node. Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        # We use the hash value of the state
        # stored in the node instead of the node
        # object itself to quickly search a node
        # with the same state in a Hash Table
        return hash(self.state)

In [ ]:
def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then w have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

In [ ]:
def uniform_cost_search(problem, display=False):
    """[Figure 3.14]"""
    return best_first_graph_search(problem, lambda n: n.path_cost, display)

In [ ]:
def greedy_search(problem, h=None, display=False):
  h = memoize(h or problem.h, 'h')
  return best_first_graph_search(problem, lambda n: h(n), display)

In [ ]:
def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)


In [ ]:
class CuboRubik(Problem):
    
    def __init__(self, initial, goal=(("","W","W","W","W","G","G","G","G","Y","Y","Y","Y","B","B","B","B","R","R","R","R","O","O","O","O"))):
      
        """ El objetivo es dejar ... """
        super().__init__(initial, goal)

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only four possible actions
        in any given state of the environment """

        possible_actions = ["U","UI","D","DI","R","RI","L","LI","F","FI","B","BI"]
        if state[0] != "":
          possible_actions.remove(state[0])
        
        #TODO: Definir acciones
        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """

        new_state = list(state)
        #TODO: Definir el resultado de las acciones. 
        do_action = { 
                    'U': ("UI",state[5],state[6],state[3],state[4]
                          
                             ,state[9],state[10],state[7],state[8]
                          
                             ,state[13],state[14],state[11],state[12]

                             ,state[1],state[2],state[15],state[16]

                             ,state[19],state[17],state[20],state[18]

                             ,state[21],state[22],state[23],state[24]),
                    
                    'UI': ("U",state[13],state[14],state[3],state[4]
                          
                             ,state[1],state[2],state[7],state[8]
                          
                             ,state[5],state[6],state[11],state[12]

                             ,state[9],state[10],state[15],state[16]

                             ,state[18],state[20],state[17],state[19]

                             ,state[21],state[22],state[23],state[24]),
                     
                    'D': ("DI",state[1],state[2],state[15],state[16]
                          
                             ,state[5],state[6],state[3],state[4]
                          
                             ,state[9],state[10],state[7],state[8]

                             ,state[13],state[14],state[11],state[12]

                             ,state[17],state[18],state[19],state[20]

                             ,state[23],state[21],state[24],state[22]),
                     
                    'DI': ("D",state[1],state[2],state[7],state[8]
                          
                             ,state[5],state[6],state[11],state[12]
                          
                             ,state[9],state[10],state[15],state[16]

                             ,state[13],state[14],state[3],state[4]

                             ,state[17],state[18],state[19],state[20]

                             ,state[22],state[24],state[21],state[23]),
                     
                     'R': ("RI",state[1],state[22],state[3],state[24]
                          
                             ,state[7],state[5],state[8],state[6]
                          
                             ,state[20],state[10],state[18],state[12]

                             ,state[13],state[14],state[15],state[16]

                             ,state[17],state[2],state[19],state[4]

                             ,state[21],state[11],state[23],state[9]),

                     'RI': ("R",state[1],state[18],state[3],state[20]
                          
                             ,state[6],state[8],state[5],state[7]
                          
                             ,state[24],state[10],state[22],state[12]

                             ,state[13],state[14],state[15],state[16]

                             ,state[17],state[9],state[19],state[11]

                             ,state[21],state[2],state[23],state[4]),

                     'L': ("LI",state[17],state[2],state[19],state[4]
                          
                             ,state[5],state[6],state[7],state[8]
                          
                             ,state[9],state[23],state[11],state[21]

                             ,state[13],state[15],state[16],state[14]

                             ,state[12],state[18],state[10],state[20]

                             ,state[1],state[22],state[3],state[24]),

                     'LI': ("L",state[21],state[2],state[23],state[4]
                          
                             ,state[5],state[6],state[7],state[8]
                          
                             ,state[9],state[19],state[11],state[17]

                             ,state[14],state[16],state[13],state[15]

                             ,state[1],state[18],state[3],state[20]

                             ,state[12],state[22],state[10],state[24]),


                     'F': ("FI",state[3],state[1],state[4],state[2]
                          
                             ,state[19],state[6],state[20],state[8]
                          
                             ,state[9],state[10],state[11],state[12]

                             ,state[13],state[21],state[15],state[22]

                             ,state[17],state[18],state[16],state[14]

                             ,state[7],state[5],state[23],state[24]),

                     'FI': ("F",state[2],state[4],state[1],state[3]
                          
                             ,state[22],state[6],state[21],state[8]
                          
                             ,state[9],state[10],state[11],state[12]

                             ,state[13],state[20],state[15],state[19]

                             ,state[17],state[18],state[5],state[7]

                             ,state[14],state[16],state[23],state[24]),

                     'B': ("BI",state[1],state[2],state[3],state[4]
                          
                             ,state[5],state[24],state[7],state[23]
                          
                             ,state[11],state[9],state[12],state[10]

                             ,state[18],state[14],state[17],state[16]

                             ,state[6],state[8],state[19],state[20]

                             ,state[21],state[22],state[13],state[15]),
                     
                     'BI': ("B",state[1],state[2],state[3],state[4]
                          
                             ,state[5],state[17],state[7],state[18]
                          
                             ,state[10],state[12],state[9],state[11]

                             ,state[23],state[14],state[24],state[16]

                             ,state[15],state[13],state[19],state[20]

                             ,state[21],state[22],state[8],state[6]),

                  
                    }
        
        new_state = do_action[action]
        
        return tuple(new_state)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """
        """Comparar arreglos sin tomar en cuenta el valor del movimiento pasado"""
        ahora = np.array(state)
        final = np.array(self.goal)
        ahora[0] = ""
        final[0] = ""

        return np.array_equal(ahora,final,False)

    def h(self, node):
        """ Return the heuristic value for a given state. Default heuristic function used is 
        h(n) = number of misplaced tiles """
       

        """Heuristica basada en colores desalineados del goal, mientras menos calzen, mayor sera el valor a retornar"""
        missPlacedColors = 0;

        
        for color in range(1, len(node.state)):
          
          if node.state[color] != self.goal[color]:
            missPlacedColors+=1
        
       
        return missPlacedColors

In [ ]:


"""SUMARIO DE MOVIMIENTOS INICIALES PARA SIMPLIFICAR TESTEO"""
#UI = ("","B","B","W","W","W","W","G","G","G","G","Y","Y","Y","Y","B","B","R","R","R","R","O","O","O","O")
#U = ("","G","G","W","W","Y","Y","G","G","B","B","Y","Y","W","W","B","B","R","R","R","R","O","O","O","O")
#D = ("","W","W","B","B","G","G","W","W","Y","Y","G","G","B","B","Y","Y","R","R","R","R","O","O","O","O")
#DI = ("","W","W","G","G","G","G","Y","Y","Y","Y","B","B","B","B","W","W","R","R","R","R","O","O","O","O")
#R = ("","W","O","W","O","G","G","G","G","R","Y","R","Y","B","B","B","B","R","W","R","W","O","Y","O","Y")
#RI = ("","W","R","W","R","G","G","G","G","O","Y","O","Y","B","B","B","B","R","Y","R","Y","O","W","O","W")
#L = ("","R","W","R","W","G","G","G","G","Y","O","Y","O","B","B","B","B","Y","R","Y","R","W","O","W","O")
#LI = ("","O","W","O","W","G","G","G","G","Y","R","Y","R","B","B","B","B","W","R","W","R","Y","O","Y","O")
#F = ("","W","W","W","W","R","G","R","G","Y","Y","Y","Y","B","O","B","O","R","R","B","B","G","G","O","O")
#FI = ("","W","W","W","W","O","G","O","G","Y","Y","Y","Y","B","R","B","R","R","R","G","G","B","B","O","O")
#B = ("","W","W","W","W","G","O","G","O","Y","Y","Y","Y","R","B","R","B","G","G","R","R","O","O","B","B")
#BI = ("","W","W","W","W","G","R","G","R","Y","Y","Y","Y","O","B","O","B","B","B","R","R","O","O","G","G")

#DIFICIL_1 = ("","B","B","B","R","O","G","G","O","Y","Y","Y","G","B","B","Y","B","R","R","R","G","O","W","O","B")
#DIFICIL_2 = ("","Y","W","B","O","G","B","B","W","R","W","B","R","O","O","Y","Y","G","Y","G","R","O","W","G","R")
#DIFICIL_3 = ("","G","G","W","O","R","Y","G","G","B","B","R","Y","W","O","B","B","R","R","W","W","O","Y","O","Y")
#MEDIANO_1(posible) = ("","R","G","W","O","R","Y","G","G","B","O","R","O","B","W","B","O","Y","R","B","W","G","Y","W","Y")


cubo = CuboRubik(("","B","B","B","R","O","G","G","O","Y","Y","Y","G","B","B","Y","B","R","R","R","G","O","W","O","B"))

print("Greedy: ")
init_time = time.time()
print (greedy_search(cubo,display=True).solution())
print (time.time()-init_time)

print("A*: ")
init_time = time.time()
print (astar_search(cubo,0,display=True).solution())
print (time.time()-init_time)

Greedy: 
